In [3]:
import requests
import urllib3
import pandas as pd
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
#https://towardsdatascience.com/using-the-strava-api-and-pandas-to-explore-your-activity-data-d94901d9bfde


In [4]:
auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

#replace variables with values for your account
payload = {
    'client_id': "107774",
    'client_secret': 'e3894bdbcf599a97f769e4964f344a7603ccc812',
    'refresh_token': 'f8c106be9364567eae147a014a25656c7ddb9855',
    'grant_type': "refresh_token",
    'f': 'json'
}

In [5]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

Requesting Token...

Access Token = 22e6f4be1cfaf682e1a80877ce552d7a58ea83e0



In [6]:
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

print(my_dataset[0]["name"])
print(my_dataset[0]["map"]["summary_polyline"])

Monday evening 5k
klxcCwujwLIDxGiAvAGM?q@G[D@FKFtAGL?DDIJoAx@cAp@g@VrDiC^[E?QLMBy@b@I?GENIpAS?E]U`@Zc@IW@aBXi@@fBUv@AF@@BGFQJq@Rp@UNIHQ?Eq@Vm@JA?zAUoAEk@@g@B~CIJD?DKFYH_ADa@C_@GO?o@Z{A\s@Fe@A


In [7]:
activities = pd.json_normalize(my_dataset)

In [29]:
#activities.columns

In [8]:
cols = ['name', 'distance', 'moving_time', 'elapsed_time','total_elevation_gain', 
          'type', 'sport_type', 'workout_type','start_date_local',
          'location_city', 'start_latlng', 'end_latlng', 'average_speed',
          'max_speed', 'upload_id']

activities = activities[cols]

activities['start_date_local'] = pd.to_datetime(activities['start_date_local'])
activities['start_time'] = activities['start_date_local'].dt.time
activities['start_date_local'] = activities['start_date_local'].dt.date

activities.head()

,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,start_date_local,location_city,start_latlng,end_latlng,average_speed,max_speed,upload_id,start_time
0,Monday evening 5k,7203.0,1954,2643,16.3,Run,Run,2,2023-05-22,None,"[21.755251558497548, 72.14996203780174]","[21.75527025014162, 72.14976883493364]",3.686,16.551,9784922370,19:58:37
1,Evening Run,1752.2,681,692,15.4,Run,Run,0,2021-11-19,None,"[21.74567892216146, 72.13510234840214]","[21.746788937598467, 72.14080673642457]",2.573,3.910,6672986713,18:15:08


In [9]:
dir = "C:\\Users\\praya\\OneDrive\\Desktop\\Personal Health Dashboard\\Strava API\\Data_exports\\"
#last_date = activities['start_date_local'].sort_values(ascending=True)[-1]
#name = f"all activities until {last_date}.csv"
activities.to_csv(dir + 'all_strava_activities.csv', mode='w', header=True, index=False)